<img src="https://theaiengineer.dev/tae_logo_gw_flatter.png" width=35% align=right>

# AI Agents & Automation — Chapter 6
## Tools and APIs

&copy; Dr. Yves J. Hilpisch<br>
AI-Powered by GPT-5.

### Overview

This notebook accompanies Chapter 6 — Tools Apis. It is self-contained and demonstrates the core ideas with small, readable code cells. Run cells from top to bottom; each code cell is preceded by a short explanation of what it does.


We create a tiny tool adapter with a timeout and retries and run a flaky tool locally.

In [ ]:
import threading  # import threading for watchdog
import time  # import time utilities

def call_with_timeout(fn, *, timeout_ms: int, **kwargs):
    """Run fn(**kwargs) with a timeout enforced by a worker thread."""
    result = {}  # store successful value
    error = {}  # store error text

    def run() -> None:
        try:
            result['value'] = fn(**kwargs)  # execute tool
        except Exception as exc:  # noqa: BLE001
            error['message'] = str(exc)  # remember failure

    worker = threading.Thread(target=run, daemon=True)  # create worker
    worker.start()  # start worker
    worker.join(timeout_ms / 1000)  # wait bounded time
    if worker.is_alive():
        return False, 'timeout'  # signal timeout
    if error:
        return False, error['message']  # propagate failure
    return True, result.get('value')  # success and payload

def flaky(counter=[0]):
    time.sleep(0.12)  # simulate slow work
    if counter[0] == 0:
        counter[0] = 1  # mark first attempt
        raise RuntimeError('transient')  # fail once
    return {'result': 'ok'}  # success payload

def call_tool(fn, timeout_ms: int = 150, max_retries: int = 1) -> dict[str, str]:
    attempts = 0  # track retries
    while True:
        attempts += 1  # increment attempt
        ok, value = call_with_timeout(fn, timeout_ms=timeout_ms)  # guarded call
        if ok:
            return {'status': 'ok', 'result': value}  # promote success
        if attempts > max_retries:
            return {'status': 'error', 'result': str(value)}  # give up
        time.sleep(0.05 * attempts)  # exponential-ish backoff

print(call_tool(flaky))  # run demo


<img src="https://theaiengineer.dev/tae_logo_gw_flatter.png" width=35% align=right>